 사람 -> 투석기, 보건소 보건소 -> 투석기, 반경 7km

 임의의 사람 점 찍기 -> 위도와 경도 확인해서 노드 찍기

 노드 타입 : 환자, 병원, 보건지소

 id : 초기화  

In [ ]:
pip install haversine

In [ ]:
import os
import folium
import pandas as pd
import numpy as np
import networkx as nx
from haversine import haversine
import seaborn as sns 
import matplotlib.pyplot as plt
plt.rc('font', family = 'NanumBarunGothic')
%matplotlib inline
from IPython.display import set_matplotlib_formats

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 투석기

In [ ]:
df_blood = pd.read_csv("/content/drive/Shareddrives/2022데이터마이닝응용/4 프로젝트/코드/주은/blood.csv", encoding = 'cp949')
df_blood.head()

,암호화요양기호,요양기관명,장비코드,장비코드명,장비대수
0,JDQ4MTg4MSM1MSMkMSMkMCMkNzIkNDgxOTYxIzIxIyQxIy...,최석진치과의원,B109,콘빔CT,1
1,JDQ4MTYyMiM4MSMkMiMkNCMkMDAkMzgxMTkxIzUxIyQxIy...,라파마취통증의학과의원,B302,초음파영상진단기,2
2,JDQ4MTYyMiM4MSMkMiMkNCMkMDAkMzgxMTkxIzUxIyQxIy...,라파마취통증의학과의원,B203,골밀도검사기,1
3,JDQ4MTYyMiM2MSMkMiMkMiMkMDAkMzgxNzAyIzIxIyQxIy...,선사영상의학과의원,B105,유방촬영장치,1
4,JDQ4MTYyMiM2MSMkMiMkMiMkMDAkMzgxNzAyIzIxIyQxIy...,선사영상의학과의원,B302,초음파영상진단기,2


In [ ]:
df_blood = df_blood[(df_blood['장비코드명']=='혈액투석을위한인공신장기') & (df_blood['장비대수']>=1)]

In [ ]:
df_blood.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1242 entries, 42 to 56257
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   암호화요양기호  1242 non-null   object
 1   요양기관명    1242 non-null   object
 2   장비코드     1242 non-null   object
 3   장비코드명    1242 non-null   object
 4   장비대수     1242 non-null   int64 
dtypes: int64(1), object(4)
memory usage: 58.2+ KB


In [ ]:
df_blood = df_blood[['암호화요양기호', '요양기관명', '장비코드명', '장비대수',]]

In [ ]:
df_blood.head()

,암호화요양기호,요양기관명,장비코드명,장비대수
42,JDQ4MTYyMiM2MSMkMSMkNiMkODkkMzgxMzUxIzExIyQxIy...,의료법인영경의료재단전주병원,혈액투석을위한인공신장기,45
130,JDQ4MTYyMiM1MSMkMSMkMCMkOTkkNTgxMzUxIzQxIyQxIy...,시민요양병원,혈액투석을위한인공신장기,11
204,JDU4MTI3MSM1MSMkMiMkMCMkMDAkMzgxOTYxIzQxIyQxIy...,21C프라자내과의원,혈액투석을위한인공신장기,22
295,JDQ4MTYyMiM4MSMkMiMkMCMkMDAkNDgxOTYxIzExIyQxIy...,윤정수연합내과의원,혈액투석을위한인공신장기,32
364,JDU4MTI3MSM1MSMkMiMkMCMkMDAkNDgxOTYxIzIxIyQxIy...,정언내과의원,혈액투석을위한인공신장기,38


In [ ]:
df2 = pd.read_csv("/content/drive/Shareddrives/2022데이터마이닝응용/4 프로젝트/코드/1. 병원정보서비스 2022.3.csv", encoding = 'cp949')
df2.head()

,암호화요양기호,요양기관명,종별코드,종별코드명,시도코드,시도코드명,시군구코드,시군구코드명,읍면동,우편번호,...,치과일반의 인원수,치과인턴 인원수,치과레지던트 인원수,치과전문의 인원수,한방일반의 인원수,한방인턴 인원수,한방레지던트 인원수,한방전문의 인원수,x좌표,y좌표
0,JDQ4MTYyMiM1MSMkMSMkMCMkODkkMzgxMzUxIzExIyQxIy...,가톨릭대학교인천성모병원,1,상급종합,220000,인천,220003,인천부평구,NaN,0,...,1,0,0,2,0,0,0,0,126.724899,37.484831
1,JDQ4MTYyMiM1MSMkMSMkNCMkODkkMzgxMzUxIzExIyQxIy...,강릉아산병원,1,상급종합,320000,강원,320100,강릉시,NaN,25440,...,0,0,0,2,0,0,0,0,128.857841,37.818433
2,JDQ4MTg4MSM1MSMkMSMkMCMkODkkMzgxMzUxIzExIyQxIy...,강북삼성병원,1,상급종합,110000,서울,110016,종로구,NaN,3181,...,3,0,0,4,0,0,0,0,126.967750,37.568408
3,JDQ4MTg4MSM1MSMkMSMkMCMkODkkMzgxMzUxIzExIyQxIy...,건국대학교병원,1,상급종합,110000,서울,110023,광진구,NaN,5030,...,0,0,0,3,0,0,0,0,127.071828,37.540376
4,JDQ4MTYyMiM4MSMkMSMkMCMkODkkMzgxMzUxIzExIyQxIy...,경북대학교병원,1,상급종합,230000,대구,230006,대구중구,NaN,41944,...,0,0,0,1,0,0,0,0,128.604125,35.866774


In [ ]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75211 entries, 0 to 75210
Data columns (total 29 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   암호화요양기호     75211 non-null  object 
 1   요양기관명       75211 non-null  object 
 2   종별코드        75211 non-null  int64  
 3   종별코드명       75211 non-null  object 
 4   시도코드        75211 non-null  int64  
 5   시도코드명       75211 non-null  object 
 6   시군구코드       75211 non-null  int64  
 7   시군구코드명      75211 non-null  object 
 8   읍면동         29094 non-null  object 
 9   우편번호        75211 non-null  int64  
 10  주소          75211 non-null  object 
 11  전화번호        74839 non-null  object 
 12  병원URL       8951 non-null   object 
 13  개설일자        75211 non-null  object 
 14  총의사수        75211 non-null  int64  
 15  의과일반의 인원수   75211 non-null  int64  
 16  의과인턴 인원수    75211 non-null  int64  
 17  의과레지던트 인원수  75211 non-null  int64  
 18  의과전문의 인원수   75211 non-null  int64  
 19  치과일반의 인원수   75211 non-nul

In [ ]:
df_step1 = df2[['암호화요양기호', '시도코드명', 'x좌표', 'y좌표']]

In [ ]:
df_step1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75211 entries, 0 to 75210
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   암호화요양기호  75211 non-null  object 
 1   시도코드명    75211 non-null  object 
 2   x좌표      73295 non-null  float64
 3   y좌표      73295 non-null  float64
dtypes: float64(2), object(2)
memory usage: 2.3+ MB


In [ ]:
df_blood = pd.merge(left = df_blood, right = df_step1, how = 'inner', on = '암호화요양기호')

In [ ]:
df_blood = df_blood.dropna()

In [ ]:
df_blood['type'] = '병원' 

In [ ]:
df_blood.head()

,암호화요양기호,요양기관명,장비코드명,장비대수,시도코드명,x좌표,y좌표,type
0,JDQ4MTYyMiM2MSMkMSMkNiMkODkkMzgxMzUxIzExIyQxIy...,의료법인영경의료재단전주병원,혈액투석을위한인공신장기,45,전북,127.124812,35.815245,병원
1,JDQ4MTYyMiM1MSMkMSMkMCMkOTkkNTgxMzUxIzQxIyQxIy...,시민요양병원,혈액투석을위한인공신장기,11,경기,126.737238,37.535658,병원
2,JDU4MTI3MSM1MSMkMiMkMCMkMDAkMzgxOTYxIzQxIyQxIy...,21C프라자내과의원,혈액투석을위한인공신장기,22,경기,127.132210,37.440727,병원
3,JDQ4MTYyMiM4MSMkMiMkMCMkMDAkNDgxOTYxIzExIyQxIy...,윤정수연합내과의원,혈액투석을위한인공신장기,32,대구,128.557938,35.943972,병원
4,JDU4MTI3MSM1MSMkMiMkMCMkMDAkNDgxOTYxIzIxIyQxIy...,정언내과의원,혈액투석을위한인공신장기,38,인천,126.707099,37.443507,병원


In [ ]:
df_blood = df_blood[df_blood['시도코드명'] == '충북']
df_blood

,암호화요양기호,요양기관명,장비코드명,장비대수,시도코드명,x좌표,y좌표,type
8,JDQ4MTYyMiM1MSMkMSMkOCMkOTkkNTgxMzUxIzExIyQxIy...,청주시립요양병원,혈액투석을위한인공신장기,17,충북,127.481779,36.596119,병원
25,JDQ4MTYyMiM1MSMkMSMkOCMkOTkkNTgxMzUxIzExIyQxIy...,의료법인정석의료재단제천하나요양병원,혈액투석을위한인공신장기,10,충북,128.209808,37.137903,병원
67,JDQ4MTYyMiM1MSMkMSMkOCMkOTkkNTgxMzUxIzExIyQxIy...,단양군립노인요양병원,혈액투석을위한인공신장기,10,충북,128.368776,36.987116,병원
83,JDQ4MTYyMiM1MSMkMSMkOCMkODkkMzgxMzUxIzExIyQxIy...,의료법인 자산의료재단 제천서울병원,혈액투석을위한인공신장기,33,충북,128.206235,37.138611,병원
195,JDQ4MTYyMiM1MSMkMiMkOCMkMDAkMzgxMzUxIzIxIyQxIy...,제중내과의원,혈액투석을위한인공신장기,31,충북,127.930370,36.973358,병원
198,JDQ4MTYyMiM1MSMkMSMkOCMkOTkkMzgxMzUxIzExIyQxIy...,의료법인 조윤의료재단 영동병원,혈액투석을위한인공신장기,28,충북,127.780894,36.185237,병원
213,JDQ4MTYyMiM1MSMkMiMkOCMkMDAkMzgxMTkxIzIxIyQxIy...,최지욱내과의원,혈액투석을위한인공신장기,31,충북,127.465047,36.634814,병원
230,JDQ4MTYyMiM1MSMkMSMkOCMkOTkkMzgxMzUxIzExIyQyIy...,의료법인 여덕의료재단 증평병원,혈액투석을위한인공신장기,13,충북,127.575409,36.777487,병원
253,JDQ4MTYyMiM1MSMkMiMkOCMkMDAkMzgxMTkxIzIxIyQyIy...,미래연합의원,혈액투석을위한인공신장기,15,충북,127.917289,36.982031,병원
397,JDQ4MTYyMiM1MSMkMSMkOCMkOTkkNTgxMzUxIzExIyQxIy...,의료법인원광의료재단청주원광효도요양병원,혈액투석을위한인공신장기,24,충북,127.452907,36.636315,병원


# 보건

In [ ]:
df_all = pd.read_csv("/content/drive/Shareddrives/2022데이터마이닝응용/4 프로젝트/코드/1. 병원정보서비스 2022.3.csv", encoding = 'cp949')
df_all.head()

,암호화요양기호,요양기관명,종별코드,종별코드명,시도코드,시도코드명,시군구코드,시군구코드명,읍면동,우편번호,...,치과일반의 인원수,치과인턴 인원수,치과레지던트 인원수,치과전문의 인원수,한방일반의 인원수,한방인턴 인원수,한방레지던트 인원수,한방전문의 인원수,x좌표,y좌표
0,JDQ4MTYyMiM1MSMkMSMkMCMkODkkMzgxMzUxIzExIyQxIy...,가톨릭대학교인천성모병원,1,상급종합,220000,인천,220003,인천부평구,NaN,0,...,1,0,0,2,0,0,0,0,126.724899,37.484831
1,JDQ4MTYyMiM1MSMkMSMkNCMkODkkMzgxMzUxIzExIyQxIy...,강릉아산병원,1,상급종합,320000,강원,320100,강릉시,NaN,25440,...,0,0,0,2,0,0,0,0,128.857841,37.818433
2,JDQ4MTg4MSM1MSMkMSMkMCMkODkkMzgxMzUxIzExIyQxIy...,강북삼성병원,1,상급종합,110000,서울,110016,종로구,NaN,3181,...,3,0,0,4,0,0,0,0,126.967750,37.568408
3,JDQ4MTg4MSM1MSMkMSMkMCMkODkkMzgxMzUxIzExIyQxIy...,건국대학교병원,1,상급종합,110000,서울,110023,광진구,NaN,5030,...,0,0,0,3,0,0,0,0,127.071828,37.540376
4,JDQ4MTYyMiM4MSMkMSMkMCMkODkkMzgxMzUxIzExIyQxIy...,경북대학교병원,1,상급종합,230000,대구,230006,대구중구,NaN,41944,...,0,0,0,1,0,0,0,0,128.604125,35.866774


In [ ]:
df_all = df_all[['암호화요양기호','요양기관명', '종별코드명', '시도코드명', '시군구코드명', 'x좌표', 'y좌표']]
df_all.head()

,암호화요양기호,요양기관명,종별코드명,시도코드명,시군구코드명,x좌표,y좌표
0,JDQ4MTYyMiM1MSMkMSMkMCMkODkkMzgxMzUxIzExIyQxIy...,가톨릭대학교인천성모병원,상급종합,인천,인천부평구,126.724899,37.484831
1,JDQ4MTYyMiM1MSMkMSMkNCMkODkkMzgxMzUxIzExIyQxIy...,강릉아산병원,상급종합,강원,강릉시,128.857841,37.818433
2,JDQ4MTg4MSM1MSMkMSMkMCMkODkkMzgxMzUxIzExIyQxIy...,강북삼성병원,상급종합,서울,종로구,126.967750,37.568408
3,JDQ4MTg4MSM1MSMkMSMkMCMkODkkMzgxMzUxIzExIyQxIy...,건국대학교병원,상급종합,서울,광진구,127.071828,37.540376
4,JDQ4MTYyMiM4MSMkMSMkMCMkODkkMzgxMzUxIzExIyQxIy...,경북대학교병원,상급종합,대구,대구중구,128.604125,35.866774


In [ ]:
df_all['종별코드명'].unique()

array(['상급종합', '종합병원', '병원', '요양병원', '정신병원', '의원', '치과병원', '치과의원', '조산원',
       '보건소', '보건지소', '보건진료소', '보건의료원', '한방병원', '한의원'], dtype=object)

In [ ]:
public_health = (df_all['시도코드명'] == '충북') & ((df_all['종별코드명'] == '보건소') | (df_all['종별코드명'] == '보건지소') | (df_all['종별코드명'] == '보건진료소') | (df_all['종별코드명'] == '보건의료원'))
df_ph = df_all[public_health]
df_ph.head()

,암호화요양기호,요양기관명,종별코드명,시도코드명,시군구코드명,x좌표,y좌표
56721,JDQ4MTYyMiM1MSMkMSMkOCMkOTIkMzgxMzUxIzExIyQxIy...,괴산군보건소,보건소,충북,괴산군,127.796687,36.807997
56743,JDQ4MTYyMiM1MSMkMSMkOCMkOTIkMzgxMzUxIzExIyQxIy...,단양군보건소,보건소,충북,단양군,128.350306,36.981614
56769,JDQ4MTYyMiM1MSMkMSMkOCMkOTIkMzgxMzUxIzExIyQxIy...,보은군보건소,보건소,충북,보은군,127.716173,36.486438
56840,JDQ4MTYyMiM1MSMkMSMkOCMkOTIkMzgxMzUxIzExIyQxIy...,영동군보건소,보건소,충북,영동군,127.775967,36.168496
56852,JDQ4MTYyMiM1MSMkMSMkOCMkOTIkMzgxMzUxIzExIyQxIy...,옥천군보건소,보건소,충북,옥천군,127.570102,36.304504


In [ ]:
df_ph['type'] = '보건소'

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df_ph.head()

,암호화요양기호,요양기관명,종별코드명,시도코드명,시군구코드명,x좌표,y좌표,type
56721,JDQ4MTYyMiM1MSMkMSMkOCMkOTIkMzgxMzUxIzExIyQxIy...,괴산군보건소,보건소,충북,괴산군,127.796687,36.807997,보건소
56743,JDQ4MTYyMiM1MSMkMSMkOCMkOTIkMzgxMzUxIzExIyQxIy...,단양군보건소,보건소,충북,단양군,128.350306,36.981614,보건소
56769,JDQ4MTYyMiM1MSMkMSMkOCMkOTIkMzgxMzUxIzExIyQxIy...,보은군보건소,보건소,충북,보은군,127.716173,36.486438,보건소
56840,JDQ4MTYyMiM1MSMkMSMkOCMkOTIkMzgxMzUxIzExIyQxIy...,영동군보건소,보건소,충북,영동군,127.775967,36.168496,보건소
56852,JDQ4MTYyMiM1MSMkMSMkOCMkOTIkMzgxMzUxIzExIyQxIy...,옥천군보건소,보건소,충북,옥천군,127.570102,36.304504,보건소


# 사람

In [ ]:
df_person = pd.read_csv("/content/drive/Shareddrives/2022데이터마이닝응용/4 프로젝트/코드/종연_코드/사람 데이터.csv")

In [ ]:
df_person.head()

,지역,x좌표,y좌표,type
0,청주,127.481918,36.698989,사람
1,청주,127.417545,36.663737,사람
2,청주,127.487583,36.589344,사람
3,청주,127.519512,36.616492,사람
4,청주,127.540307,36.613333,사람


In [ ]:
df_person['name'] = '환자'

In [ ]:
df_person.head()

,지역,x좌표,y좌표,type,name
0,청주,127.481918,36.698989,사람,환자
1,청주,127.417545,36.663737,사람,환자
2,청주,127.487583,36.589344,사람,환자
3,청주,127.519512,36.616492,사람,환자
4,청주,127.540307,36.613333,사람,환자


In [ ]:
df_person.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   지역      52 non-null     object 
 1   x좌표     52 non-null     float64
 2   y좌표     52 non-null     float64
 3   type    52 non-null     object 
 4   name    52 non-null     object 
dtypes: float64(2), object(3)
memory usage: 2.2+ KB


# 데이터 합치기

## 사람 / 투석기 병원

In [ ]:
df_blood.head()

,암호화요양기호,요양기관명,장비코드명,장비대수,시도코드명,x좌표,y좌표,type
8,JDQ4MTYyMiM1MSMkMSMkOCMkOTkkNTgxMzUxIzExIyQxIy...,청주시립요양병원,혈액투석을위한인공신장기,17,충북,127.481779,36.596119,병원
25,JDQ4MTYyMiM1MSMkMSMkOCMkOTkkNTgxMzUxIzExIyQxIy...,의료법인정석의료재단제천하나요양병원,혈액투석을위한인공신장기,10,충북,128.209808,37.137903,병원
67,JDQ4MTYyMiM1MSMkMSMkOCMkOTkkNTgxMzUxIzExIyQxIy...,단양군립노인요양병원,혈액투석을위한인공신장기,10,충북,128.368776,36.987116,병원
83,JDQ4MTYyMiM1MSMkMSMkOCMkODkkMzgxMzUxIzExIyQxIy...,의료법인 자산의료재단 제천서울병원,혈액투석을위한인공신장기,33,충북,128.206235,37.138611,병원
195,JDQ4MTYyMiM1MSMkMiMkOCMkMDAkMzgxMzUxIzIxIyQxIy...,제중내과의원,혈액투석을위한인공신장기,31,충북,127.930370,36.973358,병원


In [ ]:
df_blood = df_blood[['요양기관명','x좌표','y좌표','type']]

In [ ]:
df_blood.columns = ['NAME','longitude','latitude','TYPE']
df_blood.head()

,NAME,longitude,latitude,TYPE
8,청주시립요양병원,127.481779,36.596119,병원
25,의료법인정석의료재단제천하나요양병원,128.209808,37.137903,병원
67,단양군립노인요양병원,128.368776,36.987116,병원
83,의료법인 자산의료재단 제천서울병원,128.206235,37.138611,병원
195,제중내과의원,127.930370,36.973358,병원


In [ ]:
df_person = df_person[['name', 'x좌표', 'y좌표', 'type']]
df_person.columns = ['NAME', 'longitude', 'latitude', 'TYPE']
df_person.head()

,NAME,longitude,latitude,TYPE
0,환자,127.481918,36.698989,사람
1,환자,127.417545,36.663737,사람
2,환자,127.487583,36.589344,사람
3,환자,127.519512,36.616492,사람
4,환자,127.540307,36.613333,사람


In [ ]:
df_1 = pd.concat([df_person, df_blood], axis=0)
df_1.tail()

,NAME,longitude,latitude,TYPE
998,의료법인 정민의료재단 보은한양병원,127.718118,36.483223,병원
1031,김민옥내과의원,127.484351,36.670466,병원
1121,의료법인 정산의료재단 효성병원,127.495337,36.624479,병원
1145,신용봉내과의원,127.598395,36.989603,병원
1148,(사)대한신장복지회 대신의원,127.475475,36.635226,병원


In [ ]:
df_1.reset_index(inplace=True)
df_1['id'] = df_1.index
df_1.tail()

,index,NAME,longitude,latitude,TYPE,id
91,998,의료법인 정민의료재단 보은한양병원,127.718118,36.483223,병원,91
92,1031,김민옥내과의원,127.484351,36.670466,병원,92
93,1121,의료법인 정산의료재단 효성병원,127.495337,36.624479,병원,93
94,1145,신용봉내과의원,127.598395,36.989603,병원,94
95,1148,(사)대한신장복지회 대신의원,127.475475,36.635226,병원,95


In [ ]:
df_1 = df_1.drop('index', axis=1)

In [ ]:
df_1.head()

,NAME,longitude,latitude,TYPE,id
0,환자,127.481918,36.698989,사람,0
1,환자,127.417545,36.663737,사람,1
2,환자,127.487583,36.589344,사람,2
3,환자,127.519512,36.616492,사람,3
4,환자,127.540307,36.613333,사람,4


In [ ]:
df_1.to_csv("/content/drive/Shareddrives/2022데이터마이닝응용/4 프로젝트/코드/종연_코드/투석_최종/투석_사람_nodes.csv", index=False, encoding = 'cp949')

## 보건소, 사람, 투석기 합치기

In [ ]:
df_ph.head()

,암호화요양기호,요양기관명,종별코드명,시도코드명,시군구코드명,x좌표,y좌표,type
56721,JDQ4MTYyMiM1MSMkMSMkOCMkOTIkMzgxMzUxIzExIyQxIy...,괴산군보건소,보건소,충북,괴산군,127.796687,36.807997,보건소
56743,JDQ4MTYyMiM1MSMkMSMkOCMkOTIkMzgxMzUxIzExIyQxIy...,단양군보건소,보건소,충북,단양군,128.350306,36.981614,보건소
56769,JDQ4MTYyMiM1MSMkMSMkOCMkOTIkMzgxMzUxIzExIyQxIy...,보은군보건소,보건소,충북,보은군,127.716173,36.486438,보건소
56840,JDQ4MTYyMiM1MSMkMSMkOCMkOTIkMzgxMzUxIzExIyQxIy...,영동군보건소,보건소,충북,영동군,127.775967,36.168496,보건소
56852,JDQ4MTYyMiM1MSMkMSMkOCMkOTIkMzgxMzUxIzExIyQxIy...,옥천군보건소,보건소,충북,옥천군,127.570102,36.304504,보건소


In [ ]:
df_ph = df_ph[['요양기관명', 'x좌표', 'y좌표', 'type']]
df_ph.columns = ['NAME', 'longitude', 'latitude', 'TYPE']
df_ph.head()

,NAME,longitude,latitude,TYPE
56721,괴산군보건소,127.796687,36.807997,보건소
56743,단양군보건소,128.350306,36.981614,보건소
56769,보은군보건소,127.716173,36.486438,보건소
56840,영동군보건소,127.775967,36.168496,보건소
56852,옥천군보건소,127.570102,36.304504,보건소


In [ ]:
df_2 = pd.concat([df_1, df_ph], axis = 0)
df_2.head()
df_2.tail()

,NAME,longitude,latitude,TYPE,id
60037,하석보건진료소,127.459974,36.458762,보건소,NaN
60078,행정보건진료소,127.530698,36.536831,보건소,NaN
60086,호죽보건진료소,127.363143,36.697296,보건소,NaN
60107,화당보건진료소,127.987779,37.171856,보건소,NaN
60116,화신보건진료소,127.825626,36.149819,보건소,NaN


In [ ]:
df_2.reset_index(inplace = True)

In [ ]:
df_2.head()

,index,NAME,longitude,latitude,TYPE,id
0,0,환자,127.481918,36.698989,사람,0.0
1,1,환자,127.417545,36.663737,사람,1.0
2,2,환자,127.487583,36.589344,사람,2.0
3,3,환자,127.519512,36.616492,사람,3.0
4,4,환자,127.540307,36.613333,사람,4.0


In [ ]:
df_2['id'] = df_2.index
df_2 = df_2.drop('index', axis=1)
df_2.tail()

,NAME,longitude,latitude,TYPE,id
359,하석보건진료소,127.459974,36.458762,보건소,359
360,행정보건진료소,127.530698,36.536831,보건소,360
361,호죽보건진료소,127.363143,36.697296,보건소,361
362,화당보건진료소,127.987779,37.171856,보건소,362
363,화신보건진료소,127.825626,36.149819,보건소,363


In [ ]:
df_2.tail()

,NAME,longitude,latitude,TYPE,id
359,하석보건진료소,127.459974,36.458762,보건소,359
360,행정보건진료소,127.530698,36.536831,보건소,360
361,호죽보건진료소,127.363143,36.697296,보건소,361
362,화당보건진료소,127.987779,37.171856,보건소,362
363,화신보건진료소,127.825626,36.149819,보건소,363


In [ ]:
df_2.to_csv("/content/drive/Shareddrives/2022데이터마이닝응용/4 프로젝트/코드/종연_코드/투석_최종/투석_사람_보건소_nodes.csv", index = False, encoding='cp949')

# Link 연결

# 사람/투석

In [ ]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   NAME       96 non-null     object 
 1   longitude  96 non-null     float64
 2   latitude   96 non-null     float64
 3   TYPE       96 non-null     object 
 4   id         96 non-null     int64  
dtypes: float64(2), int64(1), object(2)
memory usage: 3.9+ KB


In [ ]:
df_link1 = pd.DataFrame(columns = ['LINK_ID', 'Source', 'Target', 'Weight'])
df_link1

,LINK_ID,Source,Target,Weight


In [ ]:
link_idx = 0
for index1, row1 in df_1[df_1['TYPE']=='사람'].iterrows():
  x1 = row1['longitude']
  y1 = row1['latitude']
  point1 = (x1,y1)
  for index1, row2 in df_1[df_1['TYPE']=='병원'].iterrows():
    x2 = row2['longitude']
    y2 = row2['latitude']
    point2 = (x2, y2)

    length = haversine(point1, point2, unit = 'km')
    # 기준은 8km
    if length <= 8 :
      df_link1.loc[link_idx] = [link_idx, row1['id'], row2['id'], length]
      link_idx += 1

df_link1.tail()

,LINK_ID,Source,Target,Weight
221,221.0,40.0,66.0,4.517606
222,222.0,40.0,94.0,7.889065
223,223.0,48.0,77.0,4.020387
224,224.0,48.0,79.0,3.713404
225,225.0,49.0,68.0,7.577889


In [ ]:
df_link1.to_csv("/content/drive/Shareddrives/2022데이터마이닝응용/4 프로젝트/코드/종연_코드/투석_최종/투석_사람_links.csv", index = False, encoding='cp949')

## 사람/투석/병원

In [ ]:
df_2.head()

,NAME,longitude,latitude,TYPE,id
0,환자,127.481918,36.698989,사람,0
1,환자,127.417545,36.663737,사람,1
2,환자,127.487583,36.589344,사람,2
3,환자,127.519512,36.616492,사람,3
4,환자,127.540307,36.613333,사람,4


In [ ]:
df_link2 = pd.DataFrame(columns = ['LINK_ID', 'Source', 'Target', 'Weight'])
df_link2

,LINK_ID,Source,Target,Weight


In [ ]:
link_idx = 0
ph_list = []
for index1, row1 in df_2[df_2['TYPE']=='사람'].iterrows():
  x1 = row1['longitude']
  y1 = row1['latitude']
  point1 = (x1,y1)
  for index1, row2 in df_2[(df_2['TYPE']=='병원') | (df_2['TYPE']=='보건소')].iterrows():
    x2 = row2['longitude']
    y2 = row2['latitude']
    point2 = (x2, y2)
    length = haversine(point1, point2, unit = 'km')
    if row2['TYPE'] == '병원' :
      if length <= 8 :
        df_link2.loc[link_idx] = [link_idx, row1['id'], row2['id'], length]
        link_idx += 1
    if row2['TYPE'] == '보건소' :
      if length <= 5 :
        df_link2.loc[link_idx] = [link_idx, row1['id'], row2['id'], length]
        link_idx += 1
        ph_list.append(row2['id'])

df_link2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 423 entries, 0 to 422
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   LINK_ID  423 non-null    float64
 1   Source   423 non-null    float64
 2   Target   423 non-null    float64
 3   Weight   423 non-null    float64
dtypes: float64(4)
memory usage: 16.5 KB


In [ ]:
for index1, row1 in df_2[df_2['TYPE']=='보건소'].iterrows():
  if row1['id'] in ph_list:
    x1 = row1['longitude']
    y1 = row1['latitude']
    point1 = (x1,y1)
    for index1, row2 in df_2[(df_2['TYPE']=='병원')].iterrows():
      x2 = row2['longitude']
      y2 = row2['latitude']
      point2 = (x2, y2)
      length = haversine(point1, point2, unit = 'km')
      # 기준은 15km
      if length <= 15 :
        link_idx += 1
        df_link2.loc[link_idx] = [link_idx, row1['id'], row2['id'], length]

df_link2.tail()

,LINK_ID,Source,Target,Weight
1225,1225.0,356.0,89.0,4.769601
1226,1226.0,356.0,90.0,6.077417
1227,1227.0,356.0,92.0,4.758572
1228,1228.0,356.0,93.0,7.723483
1229,1229.0,356.0,95.0,5.806746


In [ ]:
df_link2.to_csv("/content/drive/Shareddrives/2022데이터마이닝응용/4 프로젝트/코드/종연_코드/투석_최종/투석_사람_보건소_links.csv", index = False, encoding='cp949')